# Utilizar o openvino para inferencia

In [ ]:
from __future__ import print_function
import os
import sys

import cv2
import numpy as np
from openvino.inference_engine import IECore

In [ ]:
images_folder = "images"
images_path = [os.path.join(images_folder, image) for image in os.listdir(images_folder)]
BATCH_SIZE = 1

In [ ]:
model_xml = "converted_model/saved_model.xml"
model_bin = "converted_model/saved_model.bin"

print("Creating Inference Engine")
ie = IECore()

# Read IR
print(f"Loading network files:{model_xml, model_bin}")
net = ie.read_network(model=model_xml, weights=model_bin)

assert len(net.input_info.keys()) == 1, "Sample supports only single input topologies"
assert len(net.outputs) == 1, "Sample supports only single output topologies"

print("Preparing input blobs")
input_blob = next(iter(net.input_info))
out_blob = next(iter(net.outputs))
net.batch_size = BATCH_SIZE


In [ ]:
# Read and pre-process input images
n, c, h, w = net.input_info[input_blob].input_data.shape

def correct_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (w, h))
    image = image.transpose((2, 0, 1))  # Change data layout from HWC to CHW
    
    return image
       

images = [correct_image(image_path) for image_path in images_path]

    
print(f"Batch size is {n}")

# Loading model to the plugin
print("Loading model to the plugin")
exec_net = ie.load_network(network=net, device_name="CPU")

# Start sync inference
print("Starting inference in synchronous mode")

res = []

for image in images:
    res.append(exec_net.infer(inputs={input_blob: image})[out_blob])


# Processing output blob
print("Processing output blob")

labels_map = None

classid_str = "classid"
probability_str = "probability"
for i, probs in enumerate(res):
    probs = np.squeeze(probs)
    
    if type(probs) != list:
        probs = [probs]

    top_ind = np.argsort(probs)[-1:][::-1]
    print(f"Image {images_path[i]}\n")
    print(classid_str, probability_str)
    print("{} {}".format('-' * len(classid_str), '-' * len(probability_str)))
    for id in top_ind:
        det_label = labels_map[id] if labels_map else "{}".format(id)
        label_length = len(det_label)
        space_num_before = (len(classid_str) - label_length) // 2
        space_num_after = len(classid_str) - (space_num_before + label_length) + 2
        space_num_before_prob = (len(probability_str) - len(str(probs[id]))) // 2
        print("{}{}{}{}{:.7f}".format(' ' * space_num_before, det_label,
                                      ' ' * space_num_after, ' ' * space_num_before_prob,
                                      probs[id]))